# Import libraries and directories

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

layers = ['bronze','silver','gold']
adls = {layer: f'abfss://{layer}@carhuymdatalake.dfs.core.windows.net' for layer in layers}

adls_bronze = adls['bronze']
adls_silver = adls['silver']
adls_gold = adls['gold']

dbutils.widgets.text('date_id','DT00000')
date_id = dbutils.widgets.get('date_id')

In [0]:
print(dbutils.fs.ls(adls_bronze))
print(dbutils.fs.ls(adls_silver))
print(dbutils.fs.ls(adls_gold))

[FileInfo(path='abfss://bronze@carhuymdatalake.dfs.core.windows.net/raw_data/', name='raw_data/', size=0, modificationTime=1738151512000)]
[FileInfo(path='abfss://silver@carhuymdatalake.dfs.core.windows.net/car_sales/', name='car_sales/', size=0, modificationTime=1738171501000)]
[FileInfo(path='abfss://gold@carhuymdatalake.dfs.core.windows.net/dim_branch/', name='dim_branch/', size=0, modificationTime=1738262139000), FileInfo(path='abfss://gold@carhuymdatalake.dfs.core.windows.net/dim_date/', name='dim_date/', size=0, modificationTime=1738259033000), FileInfo(path='abfss://gold@carhuymdatalake.dfs.core.windows.net/dim_dealer/', name='dim_dealer/', size=0, modificationTime=1738258376000), FileInfo(path='abfss://gold@carhuymdatalake.dfs.core.windows.net/dim_model/', name='dim_model/', size=0, modificationTime=1738258878000), FileInfo(path='abfss://gold@carhuymdatalake.dfs.core.windows.net/fact_sales/', name='fact_sales/', size=0, modificationTime=1738262253000)]


# Data Reading

In [0]:
df = spark.read.format('parquet') \
    .option('header','True')\
    .option('inferSchema','True')\
    .load(f'{adls_bronze}/raw_data/carsales-{date_id}.parquet')

# Data Transformation

In [0]:
df = df.withColumn('model_category',split(col('Model_ID',),'-')[0])

In [0]:
df = df.withColumn('RevPerUnit',col('Revenue')/col('Units_Sold'))

# AD-Hoc

In [0]:
df.groupby('year','BranchName').agg(\
    sum('Units_Sold').alias('Total_Units_Sold')\
    ).sort('Year','Total_Units_Sold',ascending=[1,0]).display()

year,BranchName,Total_Units_Sold
2017,Alpine Motors,24
2017,Bristol Motors,23
2017,Acura Motors,23
2017,BMW Motors,23
2017,Aston Martin Motors,23
2017,Ariel Motors,21
2017,Daihatsu Motors,21
2017,Gilbern Motors,21
2017,Asia Motors Motors,20
2017,Anadol Motors,19


Databricks visualization. Run in Databricks to view.

#Data Writing

In [0]:
df.write.format('parquet') \
.mode('overwrite') \
.save(f'{adls_silver}/car_sales/{date_id}')

In [0]:

spark.sql(f'''
select * from parquet.`{adls_silver}/car_sales/{date_id}` limit 10
''').display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8621545377553699>, line 1
----> 1 spark.sql(f'''
      2 select * from parquet.`{adls_silver}/car_sales/{date_id}` limit 10
      3 ''').display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1825, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1820     else:
   1821         raise PySparkTypeError(
   1822             error_class="INVALID_TYPE",
   1823             message_parameters={"arg_name": "args", "arg_type": type(args).__name__},
  